In [1]:
##using age and experience try to find salary


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing').getOrCreate()

In [4]:
spark

In [9]:
training = spark.read.csv('Test4.csv',header=True,inferSchema=True)

In [10]:
training.show()


+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [11]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [13]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [17]:
from pyspark.ml.feature import VectorAssembler
Assembler = VectorAssembler(inputCols = ['age','Experience'],outputCol="Independent Column")

In [18]:
output = Assembler.transform(training)

In [19]:
output.show()

+---------+---+----------+------+------------------+
|     Name|age|Experience|Salary|Independent Column|
+---------+---+----------+------+------------------+
|    Krish| 31|        10| 30000|       [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|        [30.0,8.0]|
|    Sunny| 29|         4| 20000|        [29.0,4.0]|
|     Paul| 24|         3| 20000|        [24.0,3.0]|
|   Harsha| 21|         1| 15000|        [21.0,1.0]|
|  Shubham| 23|         2| 18000|        [23.0,2.0]|
+---------+---+----------+------+------------------+



In [20]:
finalized_data = output.select("Independent Column","Salary")

In [21]:
finalized_data.show()

+------------------+------+
|Independent Column|Salary|
+------------------+------+
|       [31.0,10.0]| 30000|
|        [30.0,8.0]| 25000|
|        [29.0,4.0]| 20000|
|        [24.0,3.0]| 20000|
|        [21.0,1.0]| 15000|
|        [23.0,2.0]| 18000|
+------------------+------+



In [22]:
from pyspark.ml.regression import LinearRegression 

In [23]:
##train test split
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Column',labelCol='Salary')
regressor = regressor.fit(train_data)

In [27]:
##coeficients
regressor.coefficients

DenseVector([-323.2867, 1696.8066])

In [28]:
##intercepts
regressor.intercept

22295.299605312008

In [29]:
pred_result = regressor.evaluate(test_data)

In [30]:
pred_result.predictions.show()

+------------------+------+------------------+
|Independent Column|Salary|        prediction|
+------------------+------+------------------+
|        [21.0,1.0]| 15000|17203.085755292603|
+------------------+------+------------------+

